In [1]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

df = pd.read_csv('./../Naive_Bayes/tweets/allLabeledTweets.csv')
df.head()

,message,id,tweetId,createdAt,language,inReplyToStatusId,inReplyToUserId,inReplyToScreenName,userId,userName,...,retweetedId,retweetCount,label,message_lowercase,clean_message,clean_message_no_punct,clean_message_no_stopwords_from_list,clean_message_no_punct_no_stopwords_from_list,clean_message_no_punct_no_freq_stopwords,clean_message_no_freq_stopwords
0,"Rodas sajūta, ka arī @airBaltic ir kaut kāds @...",0,1213615462581440500,2020-01-05T00:17:28+00:00,lv,NaN,NaN,NaN,62004316,Edgars Eglītis,...,NaN,0,2,"rodas sajūta, ka arī @airbaltic ir kaut kāds @...","rodas sajūta, ka arī MENTION ir kaut kāds MENT...",rodas sajūta ka arī MENTION ir kaut kāds MENT...,"rodas sajūta , MENTION kāds MENTION kaktu kant...",rodas sajūta MENTION kāds MENTION kaktu kantor...,rodas sajūta MENTION kaut MENTION kaktu kantor...,"rodas sajūta , MENTION kaut MENTION kaktu kant..."
1,"Amsterdama, @airBaltic smukulītes 👍🏻 https://...",0,1213740889476128800,2020-01-05T08:35:52+00:00,lv,NaN,NaN,NaN,213752948,Dairis Kuciks,...,NaN,0,1,"amsterdama, @airbaltic smukulītes 👍🏻 https://...","amsterdama, MENTION smukulītes 👍🏻 URL",amsterdama MENTION smukulītes 👍🏻 URL,"amsterdama , MENTION smukulītes 👍🏻 URL",amsterdama MENTION smukulītes 👍🏻 URL,amsterdama MENTION smukulītes 👍🏻 URL,"amsterdama , MENTION smukulītes 👍🏻 URL"
2,Kārtējo reizi... \r\n@airBaltic vakar raudošai...,0,1214174186069012500,2020-01-06T13:17:38+00:00,lv,NaN,NaN,NaN,1107743410646069200,Selma,...,NaN,73,2,kārtējo reizi... \r\n@airbaltic vakar raudošai...,kārtējo reizi... MENTION vakar raudošai sievie...,kārtējo reizi MENTION vakar raudošai sievie...,kārtējo reizi ... MENTION vakar raudošai sievi...,kārtējo reizi MENTION vakar raudošai sievietei...,kārtējo reizi MENTION vakar raudošai sievietei...,kārtējo reizi ... MENTION vakar raudošai sievi...
3,80.05% airBaltic akciju pieder Latvijas valsti...,0,1214176732456075300,2020-01-06T13:27:45+00:00,lv,NaN,NaN,NaN,1107743410646069200,Selma,...,NaN,2,2,80.05% airbaltic akciju pieder latvijas valsti...,NMBR % airbaltic akciju pieder latvijas valsti...,NMBR airbaltic akciju pieder latvijas valsti...,NMBR % airbaltic akciju pieder latvijas valsti...,NMBR airbaltic akciju pieder latvijas valstij ...,NMBR airbaltic akciju pieder latvijas valstij ...,NMBR % airbaltic akciju pieder latvijas valsti...
4,Laba ideja lidojumiem ar @airBaltic https://t....,0,1214197047382941700,2020-01-06T14:48:29+00:00,lv,NaN,NaN,NaN,110783755,Inga Gorbunova,...,NaN,4,0,laba ideja lidojumiem ar @airbaltic https://t....,laba ideja lidojumiem ar MENTION URL,laba ideja lidojumiem ar MENTION URL,laba ideja lidojumiem MENTION URL,laba ideja lidojumiem MENTION URL,laba ideja lidojumiem MENTION URL,laba ideja lidojumiem MENTION URL


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['label', 'data_type']).count()

message   id  tweetId  createdAt  language  \
label data_type                                               
0     train          205  205      205        205       205   
      val             37   37       37         37        37   
1     train           71   71       71         71        71   
      val             13   13       13         13        13   
2     train          103  103      103        103       103   
      val             18   18       18         18        18   

                 inReplyToStatusId  inReplyToUserId  inReplyToScreenName  \
label data_type                                                            
0     train                      0                0                    0   
      val                        0                0                    0   
1     train                      0                0                    0   
      val                        0                0                    0   
2     train                      0                0                    0   
      val                        0                0                    0   

                 userId  userName  ...  placeType  retweetedId  retweetCount  \
label data_type                    ...                                         
0     train         205       205  ...          9            0           205   
      val            37        37  ...          0            0            37   
1     train          71        71  ...          5            0            71   
      val            13        13  ...          4            0            13   
2     train         103       103  ...         10            0           103   
      val            18        18  ...          0            0            18   

                 message_lowercase  clean_message  clean_message_no_punct  \
label data_type                                                             
0     train                    205            205                     205   
      val                       37             37                      37   
1     train                     71             71                      71   
      val                       13             13                      13   
2     train                    103            103                     103   
      val                       18             18                      18   

                 clean_message_no_stopwords_from_list  \
label data_type                                         
0     train                                       205   
      val                                          37   
1     train                                        71   
      val                                          13   
2     train                                       103   
      val                                          18   

                 clean_message_no_punct_no_stopwords_from_list  \
label data_type                                                  
0     train                                                205   
      val                                                   37   
1     train                                                 71   
      val                                                   13   
2     train                                                103   
      val                                                   18   

                 clean_message_no_punct_no_freq_stopwords  \
label data_type                                             
0     train                                           205   
      val                                              37   
1     train                                            71   
      val                                              13   
2     train                                           103   
      val                                              18   

                 clean_message_no_freq_stopwords  
label data_type                                   
0     train                                  205  
      val        

In [3]:
df[df.data_type=='train']['label'].value_counts()

0    205
2    103
1     71
Name: label, dtype: int64

In [4]:
df_val = [df[df.data_type=='val'].message, df[df.data_type=='val'].label]

df_train = [df[df.data_type=='train'].message, df[df.data_type=='train'].label]
df_train = pd.concat(df_train, axis=1, keys=["message", "label"])

df_0 = df_train[df_train['label']==0]
df_1 = df_train[df_train['label']==1]
df_2 = df_train[df_train['label']==2]

df_0_downsampled = df_0.sample(df_1.shape[0])
df_2_downsampled = df_2.sample(df_1.shape[0])

df_train = pd.concat([df_0_downsampled, df_2_downsampled, df_1])

df_train['label'].value_counts()

0    71
2    71
1    71
Name: label, dtype: int64

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', 
                                          do_lower_case=True)

In [22]:
text= 'word embeddings are vectors 😀'
list_of_space_separated_pieces = text.strip().split()
ids = [tokenizer(piece, add_special_tokens=False)["input_ids"] for piece in list_of_space_separated_pieces]
unk_indices = [i for i, encoded in enumerate(ids) if tokenizer.unk_token_id in encoded]
unknown_strings = [piece for i, piece in enumerate(list_of_space_separated_pieces) if i in unk_indices]
unknown_strings

['😀']

In [25]:
unk_tokens = []
for message in df.clean_message_no_punct.values:
    list_of_space_separated_pieces = message.strip().split()
    ids = [tokenizer(piece, add_special_tokens=False)["input_ids"] for piece in list_of_space_separated_pieces]
    unk_indices = [i for i, encoded in enumerate(ids) if tokenizer.unk_token_id in encoded]
    unknown_strings = [piece for i, piece in enumerate(list_of_space_separated_pieces) if i in unk_indices]
    for unk_str in unknown_strings:
        unk_tokens.append(unk_str)

unk_tokens

['👍🏻',
 '😊',
 '😏',
 'vietā👍',
 '🇫🇷un',
 '🇱🇻',
 'krāšņumā…',
 '🙂',
 '😠',
 '😏',
 '👏',
 'n’',
 '🙈',
 'oik…',
 '🎤🤖🚀',
 '🥂🍾',
 '👍',
 '🇷🇺🤥',
 '🇱🇻',
 '😡',
 '⚽️✅',
 'rimi🇱🇻',
 'rimi🇱🇹',
 '😊',
 '🍊',
 '🙄',
 '🤬',
 '😁',
 '👍',
 '🏁',
 '🤷🏻\u200d♂️',
 '🥺un',
 '📲',
 '📲naudas',
 '📲',
 '…',
 '🎥',
 '👉',
 '⤵️',
 '🤔',
 '👍',
 '😄👍',
 'zeltu😎😜😎=tāāā',
 '🎄būs',
 '❤️',
 'bet…',
 'bet…',
 'bet…',
 '😭',
 '🤔',
 '🤕']

In [43]:
import collections

counter=collections.Counter(unk_tokens)

# print(counter)
# print(counter.values())
# print(counter.keys())
print(counter.most_common(10))
# print(dict(counter))
most_common_values= [word for word, word_count in counter.most_common(10)]
print(most_common_values)

[('👍', 3), ('bet…', 3), ('😊', 2), ('😏', 2), ('🇱🇻', 2), ('📲', 2), ('🤔', 2), ('👍🏻', 1), ('vietā👍', 1), ('🇫🇷un', 1)]
['👍', 'bet…', '😊', '😏', '🇱🇻', '📲', '🤔', '👍🏻', 'vietā👍', '🇫🇷un']


In [44]:
tokenizer.add_tokens(most_common_values, special_tokens=True)

10

In [46]:
encoded_data_train = tokenizer.batch_encode_plus(
    df_train.message.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].message.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df_train.label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [47]:
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [48]:
model.resize_token_embeddings(len(tokenizer))

Embedding(119557, 768)

In [50]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [54]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [55]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict = {0: 0, 1: 1, 2: 2,}
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [51]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(torch.device('cpu')) for b in batch)
        
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [56]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(torch.device('cpu')) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'models5/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/71 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.0338713518330749
Validation loss: 0.9223351685897164
F1 Score (Weighted): 0.6591267586929417


Epoch 2:   0%|          | 0/71 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.8085563270978524
Validation loss: 0.7785827906235404
F1 Score (Weighted): 0.6844688960471892


Epoch 3:   0%|          | 0/71 [00:00<?, ?it/s]


Epoch 3
Training loss: 0.6320491975671808
Validation loss: 0.8700829196883284
F1 Score (Weighted): 0.6226092792656259


Epoch 4:   0%|          | 0/71 [00:00<?, ?it/s]

KeyboardInterrupt: 

# Find emoji with regex

In [103]:
import re

emoji_str = "😀 😃 😄 😁 😆 😅 😂 🤣 😊 😇 🙂 🙃 😉 😌 😍 🥰 😘 😗 😙 😚 😋 😛 😝 😜 🤪 🤨 🧐 🤓 😎 🤩 🥳 😏 😒 😞 😔 😟 😕 🙁 ☹️ 😣 😖 😫 😩 🥺 😢 😭 😤 😠 😡 🤬 🤯 😳 🥵 🥶 😱 😨 😰 😥 😓 🤗 🤔 🤭 🤫 🤥 😶 😐 😑 😬 🙄 😯 😦 😧 😮 😲 🥱 😴 🤤 😪 😵 🤐 🥴 🤢 🤮 🤧 😷 🤒 🤕 😺 😸 😹 😻 😼 😽 🙀 😿 😾 💩 👋 🤚 🖐 ✋ 🖖 👌 🤏 ✌️ 🤞 🤟 🤘 🤙 👈 👉 👆 🖕 👇 ☝️ 👍 👎 ✊ 👊 🤛 🤜 👏 🙌 👐 🤲 🤝 🙏'👋🏻 🤚🏻 🖐🏻 ✋🏻 🖖🏻 👌🏻 🤏🏻 ✌🏻 🤞🏻 🤟🏻 🤘🏻 🤙🏻 👈🏻 👉🏻 👆🏻 🖕🏻 👇🏻 ☝🏻 👍🏻 👎🏻 ✊🏻 👊🏻 🤛🏻 🤜🏻 👏🏻 🙌🏻 👐🏻 🤲🏻 🙏🏻 👋🏼 🤚🏼 🖐🏼 ✋🏼 🖖🏼 👌🏼 🤏🏼 ✌🏼 🤞🏼 🤟🏼 🤘🏼 🤙🏼 👈🏼 👉🏼 👆🏼 🖕🏼 👇🏼 ☝🏼 👍🏼 👎🏼 ✊🏼 👊🏼 🤛🏼 🤜🏼 👏🏼 🙌🏼 👐🏼 🤲🏼 🙏🏼 👋🏽 🤚🏽 🖐🏽 ✋🏽 🖖🏽 👌🏽 🤏🏽 ✌🏽 🤞🏽 🤟🏽 🤘🏽 🤙🏽 👈🏽 👉🏽 👆🏽 🖕🏽 👇🏽 ☝🏽 👍🏽 👎🏽 ✊🏽 👊🏽 🤛🏽 🤜🏽 👏🏽 🙌🏽 👐🏽 🤲🏽 🙏👋🏾 🤚🏾 🖐🏾 ✋🏾 🖖🏾 👌🏾 🤏🏾 ✌🏾 🤞🏾 🤟🏾 🤘🏾 🤙🏾 👈🏾 👉🏾 👆🏾 🖕🏾 👇🏾 ☝🏾 👍🏾 👎🏾 ✊🏾 👊🏾 🤛🏾 🤜🏾 👏🏾 🙌🏾 👐🏾 🤲🏾 🙏 👋🏿 🤚🏿 🖐🏿 ✋🏿 🖖🏿 👌🏿 🤏🏿 ✌🏿 🤞🏿 🤟🏿 🤘🏿 🤙🏿 👈🏿 👉🏿 👆🏿 🖕🏿 👇🏿 ☝🏿 👍🏿 👎🏿 ✊🏿 👊🏿 🤛🏿 🤜🏿 👏🏿 🙌🏿 👐🏿 🤲🏿 🙏🏿 ❤️ 🧡 💛 💚 💙 💜 🖤 🤍 🤎 💔 ❣️ 💕 💞 💓 💗 💖 💘 💝 💟 💑🏻 💑🏼 💑🏽 💑🏾 💑🏿 💏🏻 💏🏼 💏🏽 💏🏾 💏🏿 👨🏻‍❤️‍👨🏻 👨🏻‍❤️‍👨🏼 👨🏻‍❤️‍👨🏽 👨🏻‍❤️‍👨🏾 👨🏻‍❤️‍👨🏿 👨🏼‍❤️‍👨🏻 👨🏼‍❤️‍👨🏼 👨🏼‍❤️‍👨🏽 👨🏼‍❤️‍👨🏾 👨🏼‍❤️‍👨🏿 👨🏽‍❤️‍👨🏻 👨🏽‍❤️‍👨🏼 👨🏽‍❤️‍👨🏽 👨🏽‍❤️‍👨🏾 👨🏽‍❤️‍👨🏿 👨🏾‍❤️‍👨🏻 👨🏾‍❤️‍👨🏼 👨🏾‍❤️‍👨🏽 👨🏾‍❤️‍👨🏾 👨🏾‍❤️‍👨🏿 👨🏿‍❤️‍👨🏻 👨🏿‍❤️‍👨🏼 👨🏿‍❤️‍👨🏽 👨🏿‍❤️‍👨🏾 👨🏿‍❤️‍👨🏿 👩🏻‍❤️‍👨🏻 👩🏻‍❤️‍👨🏼 👩🏻‍❤️‍👨🏽 👩🏻‍❤️‍👨🏾 👩🏻‍❤️‍👨🏿 👩🏻‍❤️‍👩🏻 👩🏻‍❤️‍👩🏼 👩🏻‍❤️‍👩🏽 👩🏻‍❤️‍👩🏾 👩🏻‍❤️‍👩🏿 👩🏼‍❤️‍👨🏻 👩🏼‍❤️‍👨🏼 👩🏼‍❤️‍👨🏽 👩🏼‍❤️‍👨🏾 👩🏼‍❤️‍👨🏿 👩🏼‍❤️‍👩🏻 👩🏼‍❤️‍👩🏼 👩🏼‍❤️‍👩🏽 👩🏼‍❤️‍👩🏾 👩🏼‍❤️‍👩🏿 👩🏽‍❤️‍👨🏻 👩🏽‍❤️‍👨🏼 👩🏽‍❤️‍👨🏽 👩🏽‍❤️‍👨🏾 👩🏽‍❤️‍👨🏿 👩🏽‍❤️‍👩🏻 👩🏽‍❤️‍👩🏼 👩🏽‍❤️‍👩🏽 👩🏽‍❤️‍👩🏾 👩🏽‍❤️‍👩🏿 👩🏾‍❤️‍👨🏻 👩🏾‍❤️‍👨🏼 👩🏾‍❤️‍👨🏽 👩🏾‍❤️‍👨🏾 👩🏾‍❤️‍👨🏿 👩🏾‍❤️‍👩🏻 👩🏾‍❤️‍👩🏼 👩🏾‍❤️‍👩🏽 👩🏾‍❤️‍👩🏾 👩🏾‍❤️‍👩🏿 👩🏿‍❤️‍👨🏻 👩🏿‍❤️‍👨🏼 👩🏿‍❤️‍👨🏽 👩🏿‍❤️‍👨🏾 👩🏿‍❤️‍👨🏿 👩🏿‍❤️‍👩🏻 👩🏿‍❤️‍👩🏼 👩🏿‍❤️‍👩🏽 👩🏿‍❤️‍👩🏾 👩🏿‍❤️‍👩🏿 🧑🏻‍❤️‍🧑🏼 🧑🏻‍❤️‍🧑🏽 🧑🏻‍❤️‍🧑🏾 🧑🏻‍❤️‍🧑🏿 🧑🏼‍❤️‍🧑🏻 🧑🏼‍❤️‍🧑🏽 🧑🏼‍❤️‍🧑🏾 🧑🏼‍❤️‍🧑🏿 🧑🏽‍❤️‍🧑🏻 🧑🏽‍❤️‍🧑🏼 🧑🏽‍❤️‍🧑🏾 🧑🏽‍❤️‍🧑🏿 🧑🏾‍❤️‍🧑🏻 🧑🏾‍❤️‍🧑🏼 🧑🏾‍❤️‍🧑🏽 🧑🏾‍❤️‍🧑🏿 🧑🏿‍❤️‍🧑🏻 🧑🏿‍❤️‍🧑🏼 🧑🏿‍❤️‍🧑🏽 🧑🏿‍❤️‍🧑🏾 👨🏻‍❤️‍💋‍👨🏻 👨🏻‍❤️‍💋‍👨🏼 👨🏻‍❤️‍💋‍👨🏽 👨🏻‍❤️‍💋‍👨🏾 👨🏻‍❤️‍💋‍👨🏿 👨🏼‍❤️‍💋‍👨🏻 👨🏼‍❤️‍💋‍👨🏼 👨🏼‍❤️‍💋‍👨🏽 👨🏼‍❤️‍💋‍👨🏾 👨🏼‍❤️‍💋‍👨🏿 👨🏽‍❤️‍💋‍👨🏻 👨🏽‍❤️‍💋‍👨🏼 👨🏽‍❤️‍💋‍👨🏽 👨🏽‍❤️‍💋‍👨🏾 👨🏽‍❤️‍💋‍👨🏿 👨🏾‍❤️‍💋‍👨🏻 👨🏾‍❤️‍💋‍👨🏼 👨🏾‍❤️‍💋‍👨🏽 👨🏾‍❤️‍💋‍👨🏾 👨🏾‍❤️‍💋‍👨🏿 👨🏿‍❤️‍💋‍👨🏻 👨🏿‍❤️‍💋‍👨🏼 👨🏿‍❤️‍💋‍👨🏽 👨🏿‍❤️‍💋‍👨🏾 👨🏿‍❤️‍💋‍👨🏿 👩🏻‍❤️‍💋‍👨🏻 👩🏻‍❤️‍💋‍👨🏼 👩🏻‍❤️‍💋‍👨🏽 👩🏻‍❤️‍💋‍👨🏾 👩🏻‍❤️‍💋‍👨🏿 👩🏻‍❤️‍💋‍👩🏻 👩🏻‍❤️‍💋‍👩🏼 👩🏻‍❤️‍💋‍👩🏽 👩🏻‍❤️‍💋‍👩🏾 👩🏻‍❤️‍💋‍👩🏿 👩🏼‍❤️‍💋‍👨🏻 👩🏼‍❤️‍💋‍👨🏼 👩🏼‍❤️‍💋‍👨🏽 👩🏼‍❤️‍💋‍👨🏾 👩🏼‍❤️‍💋‍👨🏿 👩🏼‍❤️‍💋‍👩🏻 👩🏼‍❤️‍💋‍👩🏼 👩🏼‍❤️‍💋‍👩🏽 👩🏼‍❤️‍💋‍👩🏾 👩🏼‍❤️‍💋‍👩🏿 👩🏽‍❤️‍💋‍👨🏻 👩🏽‍❤️‍💋‍👨🏼 👩🏽‍❤️‍💋‍👨🏽 👩🏽‍❤️‍💋‍👨🏾 👩🏽‍❤️‍💋‍👨🏿 👩🏽‍❤️‍💋‍👩🏻 👩🏽‍❤️‍💋‍👩🏼 👩🏽‍❤️‍💋‍👩🏽 👩🏽‍❤️‍💋‍👩🏾 👩🏽‍❤️‍💋‍👩🏿 👩🏾‍❤️‍💋‍👨🏻 👩🏾‍❤️‍💋‍👨🏼 👩🏾‍❤️‍💋‍👨🏽 👩🏾‍❤️‍💋‍👨🏾 👩🏾‍❤️‍💋‍👨🏿 👩🏾‍❤️‍💋‍👩🏻 👩🏾‍❤️‍💋‍👩🏼 👩🏾‍❤️‍💋‍👩🏽 👩🏾‍❤️‍💋‍👩🏾 👩🏾‍❤️‍💋‍👩🏿 👩🏿‍❤️‍💋‍👨🏻 👩🏿‍❤️‍💋‍👨🏼 👩🏿‍❤️‍💋‍👨🏽 👩🏿‍❤️‍💋‍👨🏾 👩🏿‍❤️‍💋‍👨🏿 👩🏿‍❤️‍💋‍👩🏻 👩🏿‍❤️‍💋‍👩🏼 👩🏿‍❤️‍💋‍👩🏽 👩🏿‍❤️‍💋‍👩🏾 👩🏿‍❤️‍💋‍👩🏿 🧑🏻‍❤️‍💋‍🧑🏼 🧑🏻‍❤️‍💋‍🧑🏽 🧑🏻‍❤️‍💋‍🧑🏾 🧑🏻‍❤️‍💋‍🧑🏿 🧑🏼‍❤️‍💋‍🧑🏻 🧑🏼‍❤️‍💋‍🧑🏽 🧑🏼‍❤️‍💋‍🧑🏾 🧑🏼‍❤️‍💋‍🧑🏿 🧑🏽‍❤️‍💋‍🧑🏻 🧑🏽‍❤️‍💋‍🧑🏼 🧑🏽‍❤️‍💋‍🧑🏾 🧑🏽‍❤️‍💋‍🧑🏿 🧑🏾‍❤️‍💋‍🧑🏻 🧑🏾‍❤️‍💋‍🧑🏼 🧑🏾‍❤️‍💋‍🧑🏽 🧑🏾‍❤️‍💋‍🧑🏿 🧑🏿‍❤️‍💋‍🧑🏻 🧑🏿‍❤️‍💋‍🧑🏼 🧑🏿‍❤️‍💋‍🧑🏽 🧑🏿‍❤️‍💋‍🧑🏾 👭 🧑‍🤝‍🧑 👬 👫 👩‍❤️‍👩 💑 👨‍❤️‍👨 👩‍❤️‍👨 👩‍❤️‍💋‍👩 💏 👨‍❤️‍💋‍👨 👩‍❤️‍💋‍👨 👪 👨‍👩‍👦 👨‍👩‍👧 👨‍👩‍👧‍👦 👨‍👩‍👦‍👦 👨‍👩‍👧‍👧 👨‍👨‍👦 👨‍👨‍👧 👨‍👨‍👧‍👦 👨‍👨‍👦‍👦 👨‍👨‍👧‍👧 👩‍👩‍👦 👩‍👩‍👧 👩‍👩‍👧‍👦 👩‍👩‍👦‍👦 👩‍👩‍👧‍👧 👨‍👦 👨‍👦‍👦 👨‍👧 👨‍👧‍👦 👨‍👧‍👧 👩‍👦 👩‍👦‍👦 👩‍👧 👩‍👧‍👦 👩‍ 👧‍👧 💋"
emoji_list = emoji_str.split(' ')
emoji_regex = '|'.join(emoji_list)
foundEmoji = re.findall(emoji_regex,text)
print(text)
print(foundEmoji)

word embeddings are vectors 😀
['😀']


In [106]:
all_emoji = []
for message in df.message:
    foundEmoji = re.findall(emoji_regex, message)
    for emoji in foundEmoji:
        all_emoji.append(emoji)

print(len(all_emoji))

29


In [108]:
counter=collections.Counter(all_emoji)

# print(counter)
# print(counter.values())
# print(counter.keys())
print(counter.most_common(15))
# print(dict(counter))
most_common_emoji= [word for word, word_count in counter.most_common(15)]
print(most_common_emoji)

[('👍', 6), ('😊', 2), ('😏', 2), ('🤔', 2), ('😎', 2), ('🙂', 1), ('😠', 1), ('👏', 1), ('🤥', 1), ('😡', 1), ('🙄', 1), ('🤬', 1), ('😁', 1), ('🥺', 1), ('👉', 1)]
['👍', '😊', '😏', '🤔', '😎', '🙂', '😠', '👏', '🤥', '😡', '🙄', '🤬', '😁', '🥺', '👉']


In [109]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', 
                                          do_lower_case=True)

tokenizer.add_tokens(most_common_values, special_tokens=True)

model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased",
                                                      num_labels=3,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

Embedding(119557, 768)

In [110]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(torch.device('cpu')) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'models5/finetuned_BERT_emoji_regex_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/71 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.110438262912589
Validation loss: 1.050589672897173
F1 Score (Weighted): 0.4947186700767264


Epoch 2:   0%|          | 0/71 [00:00<?, ?it/s]


Epoch 2
Training loss: 1.1089339944678294
Validation loss: 1.050589672897173
F1 Score (Weighted): 0.4947186700767264


...


Epoch 3
Training loss: 1.1244843887611173
Validation loss: 1.050589672897173
F1 Score (Weighted): 0.4947186700767264


Epoch 4:   0%|          | 0/71 [00:00<?, ?it/s]


Epoch 4
Training loss: 1.1054569835394201
Validation loss: 1.050589672897173
F1 Score (Weighted): 0.4947186700767264


Epoch 5:   0%|          | 0/71 [00:00<?, ?it/s]


Epoch 5
Training loss: 1.1107762707790858
Validation loss: 1.050589672897173
F1 Score (Weighted): 0.4947186700767264
